In [ ]:
import math
from collections import Counter

# Beispieldatensatz
# Die Werte sind frei erfunden und bedienen bewusst ein Klischee zum Schmunzeln
# Natürlich gibt es viele weibliche Gamer und noch mehr sportliche Gamer!
# Schau dir Schrödinger an....
data = [
    {"Alter": 18, "Geschlecht": "M", "MagSport": "Ja", "Ist-Gamer": "Ja"},
    {"Alter": 25, "Geschlecht": "M", "MagSport": "Nein", "Ist-Gamer": "Ja"},
    {"Alter": 30, "Geschlecht": "M", "MagSport": "Ja", "Ist-Gamer": "Nein"},
    {"Alter": 22, "Geschlecht": "W", "MagSport": "Nein", "Ist-Gamer": "Nein"},
    {"Alter": 27, "Geschlecht": "M", "MagSport": "Ja", "Ist-Gamer": "Nein"},
    {"Alter": 19, "Geschlecht": "W", "MagSport": "Nein", "Ist-Gamer": "Ja"},
    {"Alter": 21, "Geschlecht": "M", "MagSport": "Ja", "Ist-Gamer": "Ja"},
    {"Alter": 21, "Geschlecht": "W", "MagSport": "Nein", "Ist-Gamer": "Nein"}
]

# Funktion zur Berechnung der Gini-Impurity
def gini_impurity(data, target_attr):
    """Berechnet die Gini-Impurity einer Liste basierend auf dem Zielattribut."""
    counts = Counter(row[target_attr] for row in data)
    total = len(data)
    return 1 - sum((count / total) ** 2 for count in counts.values())

# Funktion, um kontinuierliche Werte zu splitten
def find_best_threshold(data, attr, target_attr, min_samples_split):
    """Finde den besten Schwellenwert für ein kontinuierliches Attribut."""
    sorted_data = sorted(data, key=lambda x: x[attr])
    unique_values = sorted(set(row[attr] for row in data))

    if len(unique_values) <= 1:
        return None, float('inf')

    best_gini = float('inf')
    best_threshold = None

    for i in range(len(unique_values) - 1):
        threshold = (unique_values[i] + unique_values[i + 1]) / 2
        left_split = [row for row in data if row[attr] <= threshold]
        right_split = [row for row in data if row[attr] > threshold]

        # Erweiterung für Mindest Sample Menge
        if len(left_split) < min_samples_split or len(right_split) < min_samples_split:
          continue # Überpringe Splits, die zu klein sind.

        if not left_split or not right_split:
            continue  # Überspringe leere Splits

        gini_split = (
            len(left_split) / len(data) * gini_impurity(left_split, target_attr) +
            len(right_split) / len(data) * gini_impurity(right_split, target_attr)
        )

        if gini_split < best_gini:
            best_gini = gini_split
            best_threshold = threshold

    return best_threshold, best_gini

# Entscheidungsbaum mit Gini
def id3_gini_with_continuous(data, attributes, target_attr, min_samples_split):
    """ID3-Algorithmus mit Unterstützung für kontinuierliche Attribute."""

    target_values = [row[target_attr] for row in data]
    if len(set(target_values)) == 1:  # Alle Zielwerte sind gleich
        return target_values[0]

    if not attributes:  # Keine weiteren Attribute zum Splitten
        return Counter(target_values).most_common(1)[0][0]

    # Erweiterung der Mindest-Sample-Anzahl
    if len(data) < min_samples_split:  # Überprüfe die Mindestsample-Menge
        return Counter(target_values).most_common(1)[0][0]

    best_attr = None
    best_threshold = None
    lowest_gini = float('inf')

    for attr in attributes:
        if isinstance(data[0][attr], (int, float)):  # Kontinuierliches Attribut
            threshold, gini = find_best_threshold(data, attr, target_attr, min_samples_split)
            if gini < lowest_gini:
                lowest_gini = gini
                best_attr = attr
                best_threshold = threshold
        else:  # Diskretes Attribut
            subsets = [
                [row for row in data if row[attr] == value]
                for value in set(row[attr] for row in data)
            ]
            gini = sum(
                (len(subset) / len(data)) * gini_impurity(subset, target_attr)
                for subset in subsets
            )
            if gini < lowest_gini:
                lowest_gini = gini
                best_attr = attr
                best_threshold = None

    if best_attr is None:
        return Counter(target_values).most_common(1)[0][0]

    tree = {best_attr: {}}

    if best_threshold is not None:  # Kontinuierliches Attribut
        left_split = [row for row in data if row[best_attr] < best_threshold]
        right_split = [row for row in data if row[best_attr] >= best_threshold]

        if not left_split or not right_split:
            return Counter(target_values).most_common(1)[0][0]

        tree[best_attr][f"< {best_threshold}"] = id3_gini_with_continuous(
            left_split, attributes, target_attr, min_samples_split)
        tree[best_attr][f">= {best_threshold}"] = id3_gini_with_continuous(
            right_split, attributes, target_attr, min_samples_split)
    else:  # Diskretes Attribut
        for value in set(row[best_attr] for row in data):
            subset = [row for row in data if row[best_attr] == value]
            subtree = id3_gini_with_continuous(
                subset, [attr for attr in attributes if attr != best_attr], target_attr, min_samples_split)
            tree[best_attr][value] = subtree

    return tree

def print_tree(tree, indent=""):
    """Formatiert die Ausgabe des Entscheidungsbaums."""
    if isinstance(tree, dict):
        for attr, branches in tree.items():
            for value, subtree in branches.items():
                print(f"{indent}{attr} = {value}:")
                print_tree(subtree, indent + "  ")
    else:
        print(f"{indent}--> {tree}")

def evaluate(tree, sample):
    """
    Traversiert den Entscheidungsbaum, um die Vorhersage für einen einzelnen Datenpunkt zu berechnen.
    :param tree: Der Entscheidungsbaum.
    :param sample: Der zu bewertende Datenpunkt (als Dictionary).
    :return: Die Vorhersage des Zielattributs.
    """
    if not isinstance(tree, dict):  # Wenn wir ein Blatt erreicht haben
        return tree

    for attr, branches in tree.items():
        if attr in sample:
            for condition, subtree in branches.items():
                if condition.startswith("< "):
                    threshold = float(condition.split("< ")[1])
                    if sample[attr] < threshold:
                        return evaluate(subtree, sample)
                elif condition.startswith(">= "):
                    threshold = float(condition.split(">= ")[1])
                    if sample[attr] >= threshold:
                        return evaluate(subtree, sample)
                elif sample[attr] == condition:  # Diskrete Werte
                    return evaluate(subtree, sample)
    return None  # Falls keine Bedingung erfüllt wird (kann bei unbekannten Werten passieren)


# Entscheidungsbaum erstellen
target_attr = "Ist-Gamer"
attributes = [attr for attr in data[0].keys() if attr != target_attr]
decision_tree = id3_gini_with_continuous(data, attributes, target_attr, 2)


# Entscheidungsbaum anzeigen
print("Entscheidungsbaum:")
print_tree(decision_tree)

# Beispielaufruf
new_sample = {"Alter": 23, "Geschlecht": "M", "MagSport": "Ja"}
prediction = evaluate(decision_tree, new_sample)
print(f"Vorhersage für {new_sample}: {prediction}")


Entscheidungsbaum:
Alter = < 20.0:
  --> Ja
Alter = >= 20.0:
  Alter = < 26.0:
    Geschlecht = M:
      --> Ja
    Geschlecht = W:
      --> Nein
  Alter = >= 26.0:
    --> Nein
Vorhersage für {'Alter': 23, 'Geschlecht': 'M', 'MagSport': 'Ja'}: Ja
